# 이미지 크롭하여 저장

In [2]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [25]:
# 파일 이름 숫자 설정
count = 1

# 파일 실행 후 결과 이미지 저장할 경로
mask_path = './masked_img/'

# mask_path로 지정한 폴더가 없을 시 폴더 생성
if not os.path.isdir(mask_path):
    os.mkdir('./masked_img')

In [26]:
def lane(img):
    global count
    ## 파일 불러오기
    img = cv2.imread(img, cv2.IMREAD_COLOR)
    img2 = img.copy()
    
    ## 차선 부분 추출
    # BGR -> HSV 영역 전환
    img2_hsv = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
    
    # 색 범위 지정
    low = (0, 0, 200)
    up = (250, 90, 250)
    
    img2_mask = cv2.inRange(img2_hsv, low, up)
    img2_result = cv2.bitwise_and(img2, img, mask = img2_mask)

    # 관심영역 설정
    region = np.array([[(200, 1080),
                                     (900, 400),
                                     (1100, 400),                                
                                    (1920, 1080)]], dtype = np.int64)

    # 까만 배경
    mask = np.zeros_like(img)
    
    # 구역 설정
    cv2.fillPoly(mask, region, 255)
    
    # 최종 이미지
    masked_img = cv2.bitwise_and(img2_result, mask)
    
    # 이미지 파일 저장
    cv2.imwrite(mask_path+str(count) + '.jpg', masked_img)
    
    count = count + 1

In [28]:
for i in range(1, 13):
    path = './img/img' + str(i)+'/'
    for j in os.listdir(path):
        lane(path+j)

# 허프변환

In [111]:
"""

허프변환 기본, linesp가 훨씬 정확하므로 일단 주석처리

src = cv2.imread("./day_img/day_example4.jpg")
dst = src.copy()
blur = cv2.GaussianBlur(dst, (1, 3), 0)
canny = cv2.Canny(blur, 10, 100)
lines = cv2.HoughLines(canny, 0.8, np.pi, 150, srn = 100, stn = 200, min_theta = 10, max_theta = 20)

for i in lines:
    rho, theta = i[0][0], i[0][1]
    a, b = np.cos(theta), np.sin(theta)
    x0, y0 = a*rho, b*rho

    scale = src.shape[0] + src.shape[1]

    x1 = int(x0 + scale * -b)
    y1 = int(y0 + scale * a)
    x2 = int(x0 - scale * -b)
    y2 = int(y0 - scale * a)

    cv2.line(dst, (x1, y1), (x2, y2), (0, 0, 255), 2)
    cv2.circle(dst, (x0, y0), 3, (255, 0, 0), 5, cv2.FILLED)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""

'\n\n허프변환 기본, linesp가 훨씬 정확하므로 일단 주석처리\n\nsrc = cv2.imread("./day_img/day_example4.jpg")\ndst = src.copy()\nblur = cv2.GaussianBlur(dst, (1, 3), 0)\ncanny = cv2.Canny(blur, 10, 100)\nlines = cv2.HoughLines(canny, 0.8, np.pi, 150, srn = 100, stn = 200, min_theta = 10, max_theta = 20)\n\nfor i in lines:\n    rho, theta = i[0][0], i[0][1]\n    a, b = np.cos(theta), np.sin(theta)\n    x0, y0 = a*rho, b*rho\n\n    scale = src.shape[0] + src.shape[1]\n\n    x1 = int(x0 + scale * -b)\n    y1 = int(y0 + scale * a)\n    x2 = int(x0 - scale * -b)\n    y2 = int(y0 - scale * a)\n\n    cv2.line(dst, (x1, y1), (x2, y2), (0, 0, 255), 2)\n    cv2.circle(dst, (x0, y0), 3, (255, 0, 0), 5, cv2.FILLED)\n\ncv2.imshow("dst", dst)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'

In [112]:
img = cv2.imread("./masked_img/1.jpg") # 이미지 로드
edges = cv2.Canny(img, 50, 300, apertureSize = 3) # canny edge

"""
추가적으로 필요한지는 의문이라 일단 주석처리

gray = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR) # gray scale
"""
"""
HoughLinesP 설명

cv2.HoughLinesP(image, rho, theta, threshold, lines = None, minLineLength = None, maxLineGap = None)
image = 불러올 이미지
rho = ρ값을 한번에 얼만큼 증가시키면서 조사할 것인지. 보통 1로 설정, 0.1~1로 설정 가능, 
theta = 단위는 라디안(각도(0~180) *  pi/180), 한번에 얼만큼 증가시키면서 조사할 것인지, 기본은 np.pi/180(0~360 설정가능) 
threshold = 인식할 임계값 설정
lines = 선분의 시작과 끝 좌표를 담고 있는 np.array형태의 데이터를 넣어야 함, None으로 설정해도 상관없음
minLineLength = 인식할 라인의 최소길이(픽셀단위)
maxLineGap = 선분간 얼마나 떨어져 있어도 인식할지 최대길이 지정(픽셀단위), 점 사이가 이 값보다 크면 다른 선으로 간주
"""

lines = cv2.HoughLinesP(edges, 1, np.pi/180, 45, lines = None, minLineLength = 20, maxLineGap = 3)
for i in range(len(lines)): # for문으로 모든 선에 HoughLinesP 적용
    for x1,y1,x2,y2 in lines[i]:
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),3)


cv2.imshow('img1',img)
cv2.waitKey(0)
cv2.destroyAllWindows()